<a href="https://colab.research.google.com/github/Abhishek-TyRnT/TyRnT/blob/master/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade tensorflow

     |████████████████████████████████| 421.8MB 41kB/s 
     |████████████████████████████████| 3.9MB 45.2MB/s 
     |████████████████████████████████| 450kB 56.4MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [2]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

dataset,info=tfds.load('imdb_reviews/subwords8k',with_info=True,as_supervised=True)

print(dataset)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteI1Y07J/imdb_reviews-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteI1Y07J/imdb_reviews-test.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteI1Y07J/imdb_reviews-unsupervised.tfrecord
Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.
{'test': <DatasetV1Adapter shapes: ((None,), ()), types: (tf.int64, tf.int64)>, 'train': <DatasetV1Adapter shapes: ((None,), ()), types: (tf.int64, tf.int64)>, 'unsupervised': <DatasetV1Adapter shapes: ((None,), ()), types: (tf.int64, tf.int64)>}


In [23]:
train,test=dataset['train'],dataset['test']

encoder=info.features['text'].encoder

for j in train.take(6):
  print(encoder.decode(j[0]))

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was cons

In [22]:
BATCH_SIZE=64
BUFFER_SIZE=1000
max=0
for j in train:
  if max<j[0].shape[0]:
    max=j[0].shape[0]
print(max)


3944


In [0]:

train_batch=train.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE,(max,()))
test_batch=test.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE,(max,()))

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow as tf
model=Sequential([
                  layers.Embedding(encoder.vocab_size,64),
                  layers.Bidirectional(layers.LSTM(64,return_sequences=True)),
                  layers.Bidirectional(layers.LSTM(32)),
                  layers.Dense(32,'relu'),
                  layers.Dropout(0.5),
                  layers.Dense(1)
                  
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])
history=model.fit(train_batch,epochs=8)

Epoch 1/8
391/391 [==============================] - 261s 667ms/step - loss: 0.6303 - accuracy: 0.5800
Epoch 2/8
391/391 [==============================] - 263s 672ms/step - loss: 0.3619 - accuracy: 0.8497
Epoch 3/8
391/391 [==============================] - 264s 674ms/step - loss: 0.2701 - accuracy: 0.9037
Epoch 4/8
391/391 [==============================] - 265s 677ms/step - loss: 0.2326 - accuracy: 0.9224
Epoch 5/8
391/391 [==============================] - 265s 678ms/step - loss: 0.1943 - accuracy: 0.9402
Epoch 6/8
391/391 [==============================] - 265s 678ms/step - loss: 0.1764 - accuracy: 0.9463
Epoch 7/8
391/391 [==============================] - 264s 675ms/step - loss: 0.1547 - accuracy: 0.9569
Epoch 8/8
391/391 [==============================] - 264s 674ms/step - loss: 0.1393 - accuracy: 0.9634


In [26]:
model.evaluate(test_batch)

    391/Unknown - 117s 300ms/step - loss: 0.4640 - accuracy: 0.8572

[0.4639881267343336, 0.85716]

In [32]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

def sample_predict(sample_pred_text, pad):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)

  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, max)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return (predictions)
review="It was one hell of a movie.The nail biting climax just brings us to the edge of the seat "

result=sample_predict(review,pad=True)
if result>0:
  print("Good")
else:
  print("Bad")

Good
